In [1]:
# Import of libraries
import numpy as np
import cv2
from IPython.display import clear_output
import time

# Starting message
print('Initial Configuration')

# Import of reference frame, augmented layer and mask
ref_frame = cv2.imread("ReferenceFrame.png")
agm_layer = cv2.imread("AugmentedLayer.png")
agm_mask = cv2.imread("AugmentedLayerMask.png")

# Shape of reference frame (And in generale of the video)
h, w = ref_frame.shape[:2]

# Reshaping of augmented layer and its mask
layer_reshaped = np.copy(agm_layer[0:h,0:w])
mask_reshaped = np.copy(agm_mask[0:h,0:w])

# Separation of the masks
mask_bottom = np.zeros(ref_frame.shape, np.uint8)
mask_bottom[int(h/2):h, 0:w] = mask_reshaped[int(h/2):h, 0:w]
mask_top = np.zeros(ref_frame.shape, np.uint8)
mask_top[0:int(h/2), 0:w] = mask_reshaped[0:int(h/2), 0:w]

# Create SURF object. You can specify params here or later.
# Here we set Hessian Threshold (major the value, less kp are saved)
hess_thresh = 4500
surf = cv2.xfeatures2d.SURF_create(hess_thresh)
kp_ref, des_ref = surf.detectAndCompute(ref_frame,None)

# Import Video
orig_video = cv2.VideoCapture("Multiple View.avi")

# Set video writer
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
fps = orig_video.get(cv2.CAP_PROP_FPS)
# N.B. we need to specify the correct width and height of the frames 
out = cv2.VideoWriter('AR_MultipleView.avi', fourcc, fps, (w,  h))

# Barycentre calcolous for bottom mask placing
# Bottom
gray_mask=cv2.cvtColor(mask_bottom, cv2.COLOR_BGR2GRAY)
returned ,binarized = cv2.threshold(gray_mask,127,255,0)
Mom = cv2.moments(binarized)
cX = (Mom["m10"] / Mom["m00"])
cY = (Mom["m01"] / Mom["m00"])
centerBottom = (int(cX),int(cY))
# Top
gray_mask=cv2.cvtColor(mask_top, cv2.COLOR_BGR2GRAY)
returned ,binarized = cv2.threshold(gray_mask,127,255,0)
Mom = cv2.moments(binarized)
cX = (Mom["m10"] / Mom["m00"])
cY = (Mom["m01"] / Mom["m00"])
centerTop = (int(cX),int(cY))

# Message of starting computing
#clear_output(wait=True)
print('Computing Video')

# Start Time
start_time = time.time()
    
while(orig_video.isOpened()):
    # Capture frame
    ret, frame = orig_video.read()
    if not ret or frame is None:
        # Clear cell output when new frame is available
        clear_output(wait=True)
        # Release the Video if ret is false
        orig_video.release()
        print("Released Video Resource")
        out.release()
        print("Saving Video")
        # Break exit the for loops
        break
    
    # SURF detection of current frame
    kp_frame, des_frame = surf.detectAndCompute(frame,None)

    # Initializing the matching algorithm
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    # Matching the descriptors
    matches = flann.knnMatch(des_ref,des_frame,k=2)
    # Keeping only good matches as per Lowe's ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    # Checking if we found the object
    MIN_MATCH_COUNT = 10
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp_ref[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp_frame[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    
        # Getting the coordinates of the corners of our query object in the train image
        H_mat, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        H_inv = np.linalg.inv(H_mat)
        
        # Warping of the mask
        warped_mask = cv2.warpPerspective(mask_reshaped, H_mat, (w, h))
        # Boolean total mask
        warped_mask_bool = np.equal(warped_mask, 255)
        
        # Dewarping of the frame to apply the augmented layer
        dewarped_frame = cv2.warpPerspective(frame, H_inv, (w, h))
        agm_frame = cv2.seamlessClone(layer_reshaped, dewarped_frame, mask_bottom, centerBottom, 0)
        agm_frame = cv2.seamlessClone(layer_reshaped, agm_frame, mask_top, centerTop, 0)
        
        # Rewarping of the agm frame
        warped_agm_frame = cv2.warpPerspective(agm_frame, H_mat, (w, h))

        # Restore previous values of the train images where the mask is black
        frame[warped_mask_bool] = warped_agm_frame[warped_mask_bool]
        
        # Write the output frame
        out.write(frame)

    else:
        print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
            
elapsed_time = time.time() - start_time
print('Time for Application of Augmented Layer')
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))


Released Video Resource
Saving Video
Time for Application of Augmented Layer


'00:00:29'